In [1]:
from src.utils import *
from src.recommenders import *
from src.metrics import *

In [2]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares

In [3]:
data = pd.read_csv('retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

c:\dev\geekbrains\venv\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [5]:
recs_train = data_train.groupby('user_id')['item_id'].unique().reset_index()
recs_train.columns=['user_id', 'actual']

recs_val = data_test.groupby('user_id')['item_id'].unique().reset_index()
recs_val.columns=['user_id', 'actual']

result = pd.DataFrame(index=['train', 'val'])

In [6]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват другие варианты
                                  aggfunc='sum', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

model = AlternatingLeastSquares(factors=5, 
                                regularization=0.01,
                                iterations=50, 
                                calculate_training_loss=True, 
                                use_gpu=False)
model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

  0%|          | 0/50 [00:00<?, ?it/s]

In [7]:
get_recommendations(1235, model=model)

[6534166, 1404121, 6534178, 6533889, 5703832]

In [8]:
get_similar_items_recommendations(1235, model=model)

[6534166, 6533889, 5703832, 1404121, 958067]

In [9]:
get_similar_users_recommendations(1235, model=model)

[6534178, 1404121, 893501, 986187, 1102959]

In [10]:
recs_train['overall'] = recs_train['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result.loc['train', 'overall'] = recs_train.apply(lambda row: precision_at_k(row['overall'], row['actual']), axis=1).mean()
recs_train['sim_items'] = recs_train['user_id'].apply(lambda x: get_similar_items_recommendations(x, model=model, N=5))
result.loc['train', 'sim_items'] = recs_train.apply(lambda row: precision_at_k(row['sim_items'], row['actual']), axis=1).mean()
recs_train['sim_users'] = recs_train['user_id'].apply(lambda x: get_similar_users_recommendations(x, model=model, N=5))
result.loc['train', 'sim_users'] = recs_train.apply(lambda row: precision_at_k(row['sim_users'], row['actual']), axis=1).mean()

In [11]:
recs_val['overall'] = recs_val['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result.loc['val', 'overall'] = recs_val.apply(lambda row: precision_at_k(row['overall'], row['actual']), axis=1).mean()
recs_val['sim_items'] = recs_val['user_id'].apply(lambda x: get_similar_items_recommendations(x, model=model, N=5))
result.loc['val', 'sim_items'] = recs_val.apply(lambda row: precision_at_k(row['sim_items'], row['actual']), axis=1).mean()
recs_val['sim_users'] = recs_val['user_id'].apply(lambda x: get_similar_users_recommendations(x, model=model, N=5))
result.loc['val', 'sim_users'] = recs_val.apply(lambda row: precision_at_k(row['sim_users'], row['actual']), axis=1).mean()

In [12]:
result.T

,train,val
overall,0.477231,0.138786
sim_items,0.398319,0.128012
sim_users,0.840816,0.285211
